In [ ]:
import numpy.random as npr
import numpy as np

import torch 
from jak.transformer import XFormer

import pandas as pd

import matplotlib.pyplot as plt
my_cmap = plt.get_cmap("plasma")


import numpy as np

In [ ]:
df = pd.read_csv("../data/train_JAK.csv")
df.head()

In [ ]:
smiles_vocab = "#()+-1234567=BCFHINOPS[]cilnors"

model = XFormer(vocab=smiles_vocab, token_dim=36, seq_length=128, lr=1e-3, device="cpu", tag="inference")
#model_path = "../parameters/xformer_x003_seed42/tag_xformer_x003_seed42_epoch299.pt"
model_path = "./parameters/xformer_test_run_seed13/tag_xformer_test_run_seed13_epoch99.pt"

model_state_dict = torch.load(model_path, map_location=model.my_device)

model.load_state_dict(model_state_dict)

In [ ]:
# encode SMILES into numpy arrays of features

num_values = len(df["Kinase_name"].unique()) * len(df["measurement_type"].unique())

x = torch.Tensor()
y = torch.Tensor()

x_dict = {}
y_dict = {}

m_dict = {"pIC50": 0 , "pKi": 1}
e_dict = {"JAK1": 0, "JAK2": 1, "JAK3": 2, "TYK2": 3}

number_enzymes = 4
with torch.no_grad():
    for ii in range(len(df)):
        my_smile = df["SMILES"][ii]
        if my_smile not in x_dict.keys():
            encoded = model.encode(my_smile).reshape(1, -1) #unsqueeze(0)
            x_dict[my_smile] = encoded
            y_dict[my_smile] = [0.0] * num_values

        index = m_dict[df["measurement_type"][ii]] * number_enzymes + e_dict[df["Kinase_name"][ii]]
        y_dict[my_smile][index] = df["measurement_value"][ii]


In [ ]:
x_dict[my_smile].shape

In [ ]:
x = torch.Tensor()
y = torch.Tensor()

for key in x_dict.keys():
    
    x = torch.cat([x, x_dict[key]])
    y = torch.cat([y, torch.tensor(y_dict[key]).reshape(1, num_values)])
    
print(x.shape, y.shape)    

In [ ]:
x.shape, y.shape

x_filepath = "../data/jakinase_x"
y_filepath = "../data/jakinase_y"

torch.save(x, f"{x_filepath}.pt")
torch.save(y, f"{y_filepath}.pt")

np.save(f"{x_filepath}.npy", x.numpy())
np.save(f"{y_filepath}.npy", y.numpy())